In [ ]:
%run scripts/python/standardize.py
%matplotlib inline
pd.set_option('max_columns',50)

In [ ]:
sd_data = pd.read_csv('_data/blds_csvs/cities/San_Diego.csv')

In [ ]:
add_year_month_day_df(sd_data,'issueddate','issued')

In [ ]:
sd_data.head(1)

In [ ]:
sd_data.to_csv('_data/San_Diego.csv')

In [23]:
sd_plot = sd_data.groupby(['permittypemapped','issuedMonth']).size().to_frame().reset_index().rename(columns={0:'tally'})

This is clearly not a comprehensive dataset. I wonder why San Diego is only able to provide data for 3 years (2010,15, and 17) after 2010.

In [ ]:
sd_plot

In [25]:
sd_tallies = tallies_by_permit(sd_plot,'permittypemapped','issuedMonth',colors,'month')

In [ ]:
sd_tallies

In [ ]:
pd.DataFrame(sd_tallies).to_csv('_data/sd_tallies.csv')

In [20]:
sd_year = sd_data.groupby(['permittypemapped','issuedYear']).size().to_frame().reset_index().rename(columns={0:'tally'})

In [ ]:
sd_year_tallies = tallies_by_permit(sd_year,'permittypemapped','issuedYear',colors,'year')

In [ ]:
pd.DataFrame(sd_year_tallies).to_csv('_data/tallies_sd_year.csv')

In [ ]:
sd_year_tallies

In [27]:
sum(sd_year.tally)

88

In [28]:
sum(sd_plot.tally)

88